In [1]:
import sys, os
import pathlib
import torch
import numpy as np
import tqdm
import networkx as nx
from matplotlib import pyplot as plt

In [2]:
import dgl
from dgl.data import register_data_args, load_data

In [3]:
reddit = dgl.data.RedditDataset()

Finished data loading.
  NumNodes: 232965
  NumEdges: 114615892
  NumFeats: 602
  NumClasses: 41
  NumTrainingSamples: 153431
  NumValidationSamples: 23831
  NumTestSamples: 55703


In [5]:
reddit.num_labels

41

In [50]:
reddit.train_mask.sum()

153431

In [15]:
reddit.features.shape

(232965, 602)

In [21]:
reddit.labels[reddit.train_mask]

array([17, 23, 22, ...,  2,  3, 13], dtype=int32)

In [ ]:
nx_reddit = reddit.graph.to_networkx().to_undirected()

In [45]:
list(nx_reddit.neighbors(1)).__len__()

154

In [52]:
nx_reddit = nx_reddit.to_undirected()

In [58]:
degree = nx_reddit.degree

In [67]:
np.mean(list(dict(degree).values()))

In [75]:
reddit.train_mask.nonzero()

(array([     1,      3,      4, ..., 232961, 232962, 232963]),)

In [86]:
(reddit.labels[1] == reddit.labels[list(nx_reddit.neighbors(1))]).mean()

0.974025974025974

In [114]:
def neighbor_same_label(graph: nx.Graph, train_mask: np.array, labels: np.array):
    """!
    :param graph: networkx graph
    """
    train_nodes = np.nonzero(train_mask)[0]
    same_label_neib_percentage = {}
    for node in tqdm(train_nodes):
        neibs = list(graph.neighbors(node))
        same_label_neib_percentage[node] = ((labels[node]==labels[neibs]).mean(), len(neibs))
    return same_label_neib_percentage

In [115]:
list(nx_reddit.neighbors(51))

[9897,
 58293,
 87647,
 231781,
 13090,
 196614,
 89091,
 59005,
 131924,
 70591,
 115604,
 51696,
 195324,
 109862,
 16415,
 149540,
 114134,
 14439,
 173772,
 52425,
 37825,
 214837,
 156767,
 92505]

In [116]:
from tqdm import tqdm
percent = neighbor_same_label(nx_reddit, reddit.train_mask, reddit.labels)

100%|██████████| 153431/153431 [00:11<00:00, 13766.59it/s]


In [125]:
from operator import itemgetter
value = np.array(list((percent.values())))
# percent_getter = itemgetter(0)
# percent_getter(value)\


In [ ]:
value[:,1].max()

In [100]:
labels = reddit.labels
node = 1
neibs = nx_reddit.neighbors(node)

(labels[node]==labels[neibs]).mean()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [103]:
nx_reddit.neighbors(node)

In [ ]:
1+1